In [ ]:
function read_federalistpapers()
    text = read("federalistpapers.txt", String)
    essays = split(text, r"FEDERALIST No.\ \d+")
    essays = essays[2:end-1] # remove the first and last elements
    essays = map(x -> split(x, "PUBLIUS")[1], essays)
    
    authors = ["Hamilton", "Madison", "Jay", "Unknown", "All"]
    author_list = []
    for i in eachindex(essays)  
        if (i >= 49 || i <= 58 || i == 62 || i == 63)
            push!(author_list, "Unknown") 
        end
        if occursin("HAMILTON", essays[i])
            push!(author_list, "Hamilton")
        elseif occursin("MADISON", essays[i])
            push!(author_list, "Madison")
        elseif occursin("JAY", essays[i])
            push!(author_list, "Jay")
        elseif occursin("MADISON, WITH HAMILTON", essays[i])
            push!(author_list, "MADISON, WITH HAMILTON")
        end

    end
    
    
    essays = map(x -> split(x, "To the People of the State of New York")[2], essays)
    essays = map(x -> split(x, "\r\n\r\n"), essays)
   
    #essays = map(x -> replace(x, r"\n"=>""), essays)
   
    
    return essays, author_list

end


essays,author_list = read_federalistpapers()

function find_essays_by_author(essays, author)
    indices = findall(x -> x == author, authors)
    return essays[indices]
end

function average_word_count(essays)
    total_word_count = 0
    for essay in essays
        words = split(essay, r"\s+")
        total_word_count += length(words)
    end
    return total_word_count/length(essays)
end

function average_word_length(essays)
    total_word_length = 0
    total_word_count = 0
    for essay in essays
        words = split(essay, r"\s+")
        for word in words
            total_word_length += length(word)
        end
        total_word_count += length(words)
    end
    return total_word_length/total_word_count
end

function average_sentence_count(essays)
    total_sentence_count = 0
    for essay in essays
        sentences = split(essay, r"[.?!]")
        total_sentence_count += length(sentences)
    end
    return total_sentence_count/length(essays)
end

function average_sentence_per_paragraph(essays)
    total_sentence_count = 0
    total_paragraph_count = 0
    for essay in essays
        paragraphs = split(essay, "\n")
        total_paragraph_count += length(paragraphs)
        for paragraph in paragraphs
            sentences = split(paragraph, r"[.?!]")
            total_sentence_count += length(sentences)
        end
    end
    return total_sentence_count/total_paragraph_count
end

function unique_word_count(essays)
    word_dict = Dict()
    for essay in essays
        words = split(essay, r"\s+")
        for word in words
            if haskey(word_dict, word)
                word_dict[word] += 1
            else
                word_dict[word] = 1
            end
        end
    end
    return word_dict
end


for author in authorSet
    author_essays = find_essays_by_author(essays, author)
    
    println(author, " wrote ", length(author_essays), " essays.")
    println("Average word count per essay: ", average_word_count(author_essays))
    println("Average word length per essay: ", average_word_length(author_essays))
    println("Average sentence count per essay: ", average_sentence_count(author_essays))
    println("Average sentence count per paragraph: ", average_sentence_per_paragraph(author_essays))
    word_dict = unique_word_count(author_essays)
    println("Unique words and their frequency:")
    for (word, count) in word_dict
        println(word, ": ", count)
    end
end





In [1]:
# TU PHAM
# Questions 1, 2, 4
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end


# Question 3
function separateIntoParagraphs(essays)
    array = Array{String}[]
    for i in eachindex(essays)
        paragraphs = split(essays[i], "\r\n\r\n")
        for j in eachindex(paragraphs)
            paragraphs[j] = replace(paragraphs[j], "\r\n" => " ")
        end
        push!(array, paragraphs)
    end
    return array
end

array = separateIntoParagraphs(essays)

# Questions 5, 6, 7, 8
function editText(text)
    editedText = lowercase(text)
    editedText = replace(editedText, r"\(\d+\)" => "")
    editedText = replace(editedText, r"\d+" => "")
    editedText = replace(editedText, "\r\n" => " ")
    editedText = replace(editedText, r"[,:;'-]"=> "")
    return editedText
end

function editParagraphs(array)
    for i in eachindex(array)
        for j in eachindex(array[i])
            array[i][j] = editText(array[i][j])
        end
    end
    return array
end

array = editParagraphs(array)

# Question 9
authorSet = ["HAMILTON", "MADISON", "JAY", "Unknown", "ALL"]

function addParagraphsByAuthor(paragraphs, authors)
    essaysByAuthor = [Array{String}[], Array{String}[], Array{String}[], Array{String}[], Array{String}[]]
    for i in eachindex(authors)
        if(cmp(authors[i], "HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
        elseif(cmp(authors[i], "MADISON") == 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "MADISON, WITH HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "JAY") == 0)
            jay = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(jay, paragraphs[i][j])
            end
            push!(essaysByAuthor[3], jay)
        else
            unknown = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(unknown, paragraphs[i][j])
            end
            push!(essaysByAuthor[4], unknown)
        end
    end
    essaysByAuthor[5] = paragraphs
    return essaysByAuthor
end

paragraphsByAuthor = addParagraphsByAuthor(array, authors)

# Question 9a
function avgWordsPerEssay(authorParagraphs)
    wordCount = 0
    for i in eachindex(authorParagraphs)
        for j in eachindex(authorParagraphs[i])
            wordCount += (count(" ", authorParagraphs[i][j]) + 1)
        end
    end
    return wordCount / length(authorParagraphs)
end

for i in 1:5
    display("Number of words per essay by $(authorSet[i]): $(avgWordsPerEssay(paragraphsByAuthor[i]))")
end

#Incomplete
#=
function avgWordLength(authorEssays)
    wordCount = avgWordsPerEssay(authorEssays) * length(authorEssays)
end
=#

function find_essays_by_author(essays, author)
    indices = findall(x -> x == author, authors)
    return essays[indices]
end

function average_word_count(essays)
    total_word_count = 0
    for essay in essays
        words = split(essay, r"\s+")
        total_word_count += length(words)
    end
    return total_word_count/length(essays)
end

function average_word_length(essays)
    total_word_length = 0
    total_word_count = 0
    for essay in essays
        words = split(essay, r"\s+")
        for word in words
            total_word_length += length(word)
        end
        total_word_count += length(words)
    end
    return total_word_length/total_word_count
end

function average_sentence_count(essays)
    total_sentence_count = 0
    for essay in essays
        sentences = split(essay, r"[.?!]")
        total_sentence_count += length(sentences)
    end
    return total_sentence_count/length(essays)
end

function average_sentence_per_paragraph(essays)
    total_sentence_count = 0
    total_paragraph_count = 0
    for essay in essays
        paragraphs = split(essay, "\n")
        total_paragraph_count += length(paragraphs)
        for paragraph in paragraphs
            sentences = split(paragraph, r"[.?!]")
            total_sentence_count += length(sentences)
        end
    end
    return total_sentence_count/total_paragraph_count
end

function unique_word_count(essays)
    word_dict = Dict()
    for essay in essays
        words = split(essay, r"\s+")
        for word in words
            if haskey(word_dict, word)
                word_dict[word] += 1
            else
                word_dict[word] = 1
            end
        end
    end
    return word_dict
end


for author in authorSet
    author_essays = find_essays_by_author(essays, author)
    
    println(author, " wrote ", length(author_essays), " essays.")
    println("Average word count per essay: ", average_word_count(author_essays))
    println("Average word length per essay: ", average_word_length(author_essays))
    println("Average sentence count per essay: ", average_sentence_count(author_essays))
    println("Average sentence count per paragraph: ", average_sentence_per_paragraph(author_essays))
    word_dict = unique_word_count(author_essays)
    println("Unique words and their frequency:")
    for (word, count) in word_dict
        println(word, ": ", count)
    end

end


# First, obtain the word count dictionaries for each author
madison_essays = find_essays_by_author(essays, "MADISON")
madison_words = unique_word_count(madison_essays)
hamilton_essays = find_essays_by_author(essays, "HAMILTON")
hamilton_words = unique_word_count(hamilton_essays)
jay_essays = find_essays_by_author(essays, "JAY")
jay_words = unique_word_count(jay_essays)

# Define the Score function
# Currently need to write into the loop above to save time
function Score(author_words, other1_words, other2_words, word)
    M = get(author_words, word, 0) / sum(values(author_words))
    H = get(other1_words, word, 0) / sum(values(other1_words))
    J = get(other2_words, word, 0) / sum(values(other2_words))
    return abs(M - H) + abs(M - J)
end

# Find the 10 words with the highest Score for each author
madison_scores = Dict()
for word in keys(madison_words)
    madison_scores[word] = Score(madison_words, hamilton_words, jay_words, word)
end
madison_top10 = sort(collect(madison_scores), by=x->x[2], rev=true)[1:10]

hamilton_scores = Dict()
for word in keys(hamilton_words)
    hamilton_scores[word] = Score(hamilton_words, madison_words, jay_words, word)
end
hamilton_top10 = sort(collect(hamilton_scores), by=x->x[2], rev=true)[1:10]

jay_scores = Dict()
for word in keys(jay_words)
    jay_scores[word] = Score(jay_words, madison_words, hamilton_words, word)
end
jay_top10 = sort(collect(jay_scores), by=x->x[2], rev=true)[1:10]

# Print the results
println("Top 10 words for Madison:")
for word in madison_top10
    println(word[1])
end

println("Top 10 words for Hamilton:")
for word in hamilton_top10
    println(word[1])
end

println("Top 10 words for Jay:")
for word in jay_top10
    println(word[1])
end

# First, obtain the word count dictionaries for each author
madison_essays = filter(e -> e[2] == "Madison", authors)
madison_words = unique_word_count([e[1] for e in madison_essays])
hamilton_essays = filter(e -> e[2] == "Hamilton", authors)
hamilton_words = unique_word_count([e[1] for e in hamilton_essays])
jay_essays = filter(e -> e[2] == "Jay", authors)
jay_words = unique_word_count([e[1] for e in jay_essays])

# Define the Score function
function Score(author_words, other1_words, other2_words, word)
    M = get(author_words, word, 0) / sum(length(author_words))
    H = get(other1_words, word, 0) / sum(values(other1_words))
    J = get(other2_words, word, 0) / sum(values(other2_words))
    return abs(M - H) + abs(M - J)
end

# Find the 10 words with the highest Score for each author
madison_scores = Dict()
for word in keys(madison_words)
    madison_scores[word] = Score(madison_words, hamilton_words, jay_words, word)
end
madison_top10 = sort(collect(madison_scores), by=x->x[2], rev=true)[1:10]

hamilton_scores = Dict()
for word in keys(hamilton_words)
    hamilton_scores[word] = Score(hamilton_words, madison_words, jay_words, word)
end
hamilton_top10 = sort(collect(hamilton_scores), by=x->x[2], rev=true)[1:10]

jay_scores = Dict()
for word in keys(jay_words)
    jay_scores[word] = Score(jay_words, madison_words, hamilton_words, word)
end
jay_top10 = sort(collect(jay_scores), by=x->x[2], rev=true)[1:10]

# Print the results
println("Top 10 words for Madison:")
for word in madison_top10
    println(word[1])
end

println("Top 10 words for Hamilton:")
for word in hamilton_top10
    println(word[1])
end

println("Top 10 words for Jay:")
for word in jay_top10
    println(word[1])
end

"Number of words per essay by HAMILTON: 2179.823529411765"

"Number of words per essay by MADISON: 2768.5714285714284"

"Number of words per essay by JAY: 1674.2"

"Number of words per essay by Unknown: 1976.0"

"Number of words per essay by ALL: 2211.0823529411764"

Top 10 words for Madison:
the
and
of
a
to
would
by
on
is
or
Top 10 words for Hamilton:
the
and
of
to
a
in
upon
by
on
their
Top 10 words for Jay:
the
and
of
a
they
is
their
or
which
to


LoadError: BoundsError: attempt to access 0-element Vector{Pair{Any, Any}} at index [1:10]

In [ ]:
# Tester
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
println(length(essays))
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end

# println(essays[1])

In [ ]:
# Minh's implementation

# Note: the number of the essay will be index no.

# Break it up into the individual essays. Each essay starts with "Federalist No. n." Within each essay you can skip everything up to and including "To the People of the State of New York:" and the footnotes after "PUBLIUS." You can also skip everything up to the start of the first essay and everything after the end of the last essay. (3 points)
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)

from = "To the People of the State of New York:"
to = "PUBLIUS" 

# Loop over each essay and extract the text in the essays
for (i, essay) in enumerate(essays)

    search_from = findfirst(from, essay)
    search_to = findfirst(to, essay)

    # Find the indices of from and to in the essay string
    start_index = search_from[1] + length(from)
    end_index = search_to[1] - 1
    
    # Extract the text between from and to
    extracted_text = essay[start_index:end_index]

    # Remove trailing new lines in the start and the end of the needed text
    extracted_text = rstrip(extracted_text) # somehow this does not remove the last line of the last paragraph, but it create a concrete pattern for all paragraphs (to have one new line at the end), so I guess it works.
    extracted_text = lstrip(extracted_text)

    # Print the extracted text
    # println(extracted_text)
    # println(i)

    # Replace with only the needed text from each essay
    essays[i] = extracted_text 
    
end
# println(typeof(essays[1]))
println(essays[1])

